In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## Find Local Alignment Windows
Let $n = |v|, m = |w|$, assume $n << m$.

First we will find a list of all quartets $(i_{begin}, j_{begin}, i_{end}, j_{end})$ of all optimal
local alignments of $v$ with $w$. Since this quartet captures the 4 points of a "window" of what would
be the 2D DP table for local alignment, I'll call it a window.

This is extremely similar to the midterm! Professor El-Kebir pointed out a clever solution to this
problem by keeping track of an `origin` "table" where each cell $(i, j)$ has an entry $origin[i][j]$
corresponding to the start of the optimal local alignment ending at $(i, j)$. We run space
efficient local alignment with 2 columns (and 2 columns for our `origin` tracking), and
save the highest scores + their corrseponding windows. There may be multiple ties by score.

These windows will
then be plugged into Hirschberg's algorithm to be able to figure out the backtraces for the local
aligns in $O(n)$ space.


In [2]:
def delta(v_i, w_j):
    if v_i == w_j:
        return 1
    else:
        return -1


def find_windows(v: str, w: str):
    n = len(v)
    m = len(w)
    prev = [(0, -1, -1) for _ in range(n + 1)]  # (score, i, j) i and j of origin of score
    cur = [(0, -1, -1) for _ in range(n + 1)]
    max_score = 0
    windows = []

    for j_end in range(m + 1):
        for i_end in range(n + 1):
            new_score, i_beg, j_beg = 0, i_end, j_end  # origin is itself by default
            if i_end > 0:
                from_score, from_i_beg, from_j_beg = cur[i_end - 1]
                deletion_score = from_score - 1
                if deletion_score > new_score:
                    # inherit origins from previous cell
                    new_score, i_beg, j_beg = deletion_score, from_i_beg, from_j_beg
            if j_end > 0:
                from_score, from_i_beg, from_j_beg = prev[i_end]
                insertion_score = from_score - 1
                if insertion_score > new_score:
                    new_score, i_beg, j_beg = insertion_score, from_i_beg, from_j_beg
            if i_end > 0 and j_end > 0:
                from_score, from_i_beg, from_j_beg = prev[i_end - 1]
                match_score = from_score + delta(v[i_end - 1], w[j_end - 1])
                if match_score > new_score:
                    new_score, i_beg, j_beg = match_score, from_i_beg, from_j_beg

            cur[i_end] = (new_score, i_beg, j_beg)
            if new_score > max_score:
                max_score = new_score
                # print(f'new window of score {max_score}')
                # window index is 0-indexed for string access, but sequence index is 1-indexed
                windows = [(i_beg, j_beg, i_end, j_end)]
            elif new_score == max_score and max_score > 0:
                # print(f'continuing window of score {max_score}')
                windows.append((i_beg, j_beg, i_end, j_end))

        prev = cur
        cur = [(0, -1, -1) for _ in range(n + 1)]
        # print([p[0] for p in prev])

    # print([c[0] for c in cur])
    return max_score, windows

In [3]:
# Test finding windows
test_in = 'GTAAATCCTTTGAGAAGAAGAGTCTCT'
test_seq = 'GTTTAATGATTCACGATGTTGAGCACAGTTTTCCAACATTATGACCGAAATGATGAGGAACGCGCGTTGGTACCCTATAATCCGAGGCCGCCGAGTTACG'
print(f'{len(test_in)}: {test_in}')
print(f'{len(test_seq)}: {test_seq}')

27: GTAAATCCTTTGAGAAGAAGAGTCTCT
100: GTTTAATGATTCACGATGTTGAGCACAGTTTTCCAACATTATGACCGAAATGATGAGGAACGCGCGTTGGTACCCTATAATCCGAGGCCGCCGAGTTACG


In [4]:
find_windows(test_in, test_seq)

(8, [(3, 34, 22, 57)])

In [5]:
len(test_in)
print(test_in[3:21+1])
print(test_seq[34:56+1])

AATCCTTTGAGAAGAAGAG
AACATTATGACCGAAATGATGAG


## Hirschberg Implementation

Now that we have our window-finding done, here we'll implement Hirschberg's algorithm to be able
to recover the backtraces in $O(n)$ space where $n = \text{length of input sequence}$ and assuming
$n << m$.

Note that our implementation is more closely aligned with [Wikipedia's suggested implementation](https://en.wikipedia.org/wiki/Hirschberg%27s_algorithm)
than the one discussed in class.

In [6]:
# Returns score of last col only, not alignment string
def needleman_wunsch_score(v: str, w: str):
    n = len(v)
    m = len(w)
    prev = np.zeros(n + 1, dtype=np.int64)
    cur = np.zeros(n + 1, dtype=np.int64)

    for j_end in range(m + 1):
        for i_end in range(n + 1):
            new_score = -np.inf
            if i_end == 0 and j_end == 0:
                new_score = 0
            if i_end > 0:
                deletion_score = cur[i_end - 1] - 1
                if deletion_score > new_score:
                    new_score = deletion_score
            if j_end > 0:
                insertion_score = prev[i_end] - 1
                if insertion_score > new_score:
                    new_score = insertion_score
            if i_end > 0 and j_end > 0:
                match_score = prev[i_end - 1] + delta(v[i_end - 1], w[j_end - 1])
                if match_score > new_score:
                    new_score = match_score

            cur[i_end] = new_score

        prev = cur
        cur = np.zeros(n + 1, dtype=np.int64)

    return prev


def hirschberg(X: str, Y: str):
    Z = ''
    W = ''
    n = len(X)
    m = len(Y)
    if n == 0:
        Z = '-' * m
        W = Y
    elif m == 0:
        Z = X
        W = '-' * n
    elif n == 1:  # Needleman-Wunsch for single character case
        match_idx = Y.find(X)
        Z = (X + '-' * (m - 1)) if match_idx == -1 \
            else ('-' * match_idx + X + '-' * (m - match_idx - 1))
        W = Y
    elif m == 1:  # Needleman-Wunsch for single character case
        match_idx = X.find(Y)
        Z = X
        W = (Y + '-' * (n - 1)) if match_idx == -1 \
            else ('-' * match_idx + Y + '-' * (n - match_idx - 1))
    else:
        j = m // 2
        score_left = needleman_wunsch_score(X, Y[:j])
        score_right = needleman_wunsch_score(X[::-1], Y[-1:j - 1:-1])
        i = np.argmax(score_left + score_right[::-1])
        Z1, W1 = hirschberg(X[:i], Y[:j])
        Z2, W2 = hirschberg(X[i:], Y[j:])
        Z = Z1 + Z2
        W = W1 + W2

    return (Z, W)

In [7]:
# Test NW and Hirschberg
nw_in = 'ACAGTTTTCCAACATTA'
nw_seq = 'ACAGTTTTCACCATATTAGACAGAACTAGTGAGAG'
needleman_wunsch_score(nw_in, nw_seq)

array([-35, -33, -31, -29, -27, -25, -23, -21, -19, -17, -15, -13, -11,
        -9,  -7,  -5,  -3,  -1])

In [8]:
# Z, W = hirschberg('AGTACGCA', 'TATGC')
Z, W = hirschberg('ACAGTTTTCCAACATTA', 'ACAGTTTTCACCATATTAGACAGAACTAGTGAGAG')
print(Z)
print(W)

ACAGTTTT--CC-----A-ACA----T--T-A---
ACAGTTTTCACCATATTAGACAGAACTAGTGAGAG


## Putting It All Together

Almost there! All that's left is to combine these two components to get our space-efficient
local alignment.

In [9]:
def space_efficient_local_align(v: str, w: str):
    alignments = []
    score, windows = find_windows(v, w)
    for i_beg, j_beg, i_end, j_end in windows:
        alignments.append('\n'.join(hirschberg(v[i_beg:i_end], w[j_beg:j_end])))
    return score, alignments

In [10]:
# Test our new algorithm's output!
v = 'GTAAATCCTTTGAGAAGAAGAGTCTCT'
w = 'GTTTAATGATTCACGATGTTGAGCACAGTTTTCCAACATTATGACCGAAATGATGAGGAACGCGCGTTGGTACCCTATAATCCGAGGCCGCCGAGTTACG'
score, alignments = space_efficient_local_align(v, w)
print(f'Score: {score}')
print('Alignments:')
for align in alignments:
    print(align)
    print()

Score: 8
Alignments:
AATCCTT-TGA--G-AA-GAAGAG
AA-CATTATGACCGAAATGATGAG



## Using our Gizmo To Analyze Database
Now for the exciting part, doing our mini-BLAST search!

## Loading Dataset and Performing Search
Scrubbed dataset collected by filtering through NCBI datasets (see `scrubbed-viral-data/README.md` for more info).

In [11]:
data_path = '../scrubbed-viral-data/scrubbed_sequences.csv'
df = pd.read_csv(data_path)

In [12]:
###### IMPORTANT: Input query sequence here. ######
query_seq = 'GTATTACCAATTTGGGAATTTGTTTGG'
###################################################

search_results = []  # list of (score, description, alignment) tuples
for i, row in tqdm(df.iterrows(), total=len(df)):
    score, alignments = space_efficient_local_align(query_seq, row['Sequence'])
    search_results.append((score, row['Description'], alignments))

100%|██████████| 411/411 [00:24<00:00, 16.63it/s]


In [13]:
search_results.sort(key=lambda x: x[0], reverse=True)
for score, description, alignments in search_results[:3]:
    print(f'Score: {score}, Description: {description}')
    for align in alignments:
        print()
        print(align)
    print('*' * 80)

Score: 17, Description: NC_012801.1 |Human cosavirus B1, complete genome

TAC-CAATTTGGGA-ATTTGTTT
TACTCAATTTGGGACATTGGTTT
********************************************************************************
Score: 16, Description: NC_038357.1 |Torque teno midi virus 10 DNA, complete genome, isolate: MDJN14

TATTACCAAT--TTGG-G---AATTTGTT--TGG
TATTCCCAATAATTGGAGTTAAATTTGTTAATGG
********************************************************************************
Score: 16, Description: NC_006577.2 |Human coronavirus HKU1, complete genome

TATTACCAA-TTTGG-GA--ATTTGTTTGG
TACTACTAATTTTGGTGAAGATTT-TTTGG
********************************************************************************
